In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# prefix path
prefix = '/content/drive/MyDrive/DCC/cv/task2/mini'



###### *tpu로 안되니 gpu로 할 것*

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

# 데이터 다운로드

> CIFAR-10은 32 x 32 크기의 60000개의 이미지로 이루어져 있습니다.
10개의 클래스로 분류(airplane, bird, cat ...) 되며 각각의 클래스는 6000개의 이미지로 이루어져 있습니다.
또, 5000개는 학습 데이터, 1000개는 테스트 데이터입니다
(총, 50000개의 학습 데이터, 10000개의 테스트 데이터).



In [ ]:
#데이터 셋
cifar10 = keras.datasets.cifar10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

170508288/170498071 [==============================] - 4s 0us/step


In [ ]:
# dataset shape
print(train_images.shape, test_images.shape)

(50000, 32, 32, 3) (10000, 32, 32, 3)


# 데이터 전처리


*   x는 255로 나누어 줘서 0에서 1사이로 변환시켜줌
*   y는 원 핫 인코딩을 적용함



In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(train_images, 
                                                  train_labels, 
                                                  test_size = 0.2, 
                                                  random_state=100)

In [ ]:
X_train.shape, X_val.shape

((40000, 32, 32, 3), (10000, 32, 32, 3))

In [ ]:
# the number of classes
num_class = len(np.unique(Y_val))
num_class

10

In [ ]:
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0
test_images2 = test_images.astype('float32') / 255.0

In [ ]:
# make one-hot label
Y_train = keras.utils.to_categorical(Y_train)
Y_val = keras.utils.to_categorical(Y_val)
test_labels = keras.utils.to_categorical(test_labels)

# No Argumentation 모델

In [ ]:
# with strategy.scope():
  # model
model = keras.models.Sequential([
            keras.layers.Conv2D(filters = 32, kernel_size=(5, 5), activation='relu', input_shape=(32,32,3)),
            keras.layers.MaxPool2D(pool_size=(3, 3)),
            keras.layers.Conv2D(filters = 64 , kernel_size=(3, 3), activation='relu'),
            keras.layers.MaxPool2D(pool_size=(2, 2)),
            keras.layers.Flatten(),
            keras.layers.Dense(256, activation='relu'),
            keras.layers.Dense(num_class, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics='accuracy'
)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 9, 9, 32)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               147712    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2

In [ ]:
hist = model.fit(X_train, Y_train, validation_data=(X_val,Y_val),
                 batch_size=128,
                 epochs=20)

Epoch 1/20
313/313 [==============================] - 37s 115ms/step - loss: 1.7146 - accuracy: 0.3749 - val_loss: 1.4821 - val_accuracy: 0.4668
Epoch 2/20
313/313 [==============================] - 36s 114ms/step - loss: 1.4102 - accuracy: 0.4941 - val_loss: 1.3794 - val_accuracy: 0.5071
Epoch 3/20
313/313 [==============================] - 36s 114ms/step - loss: 1.2849 - accuracy: 0.5415 - val_loss: 1.2415 - val_accuracy: 0.5609
Epoch 4/20
313/313 [==============================] - 36s 114ms/step - loss: 1.1899 - accuracy: 0.5796 - val_loss: 1.1892 - val_accuracy: 0.5773
Epoch 5/20
313/313 [==============================] - 36s 114ms/step - loss: 1.1092 - accuracy: 0.6112 - val_loss: 1.0937 - val_accuracy: 0.6163
Epoch 6/20
313/313 [==============================] - 36s 114ms/step - loss: 1.0418 - accuracy: 0.6348 - val_loss: 1.0638 - val_accuracy: 0.6308
Epoch 7/20
313/313 [==============================] - 36s 115ms/step - loss: 0.9787 - accuracy: 0.6571 - val_loss: 1.0439 - val_ac

In [ ]:
model.evaluate(test_images, test_labels)

313/313 [==============================] - 3s 10ms/step - loss: 294.0342 - accuracy: 0.5417


[294.03424072265625, 0.541700005531311]

# 데이터 Argumentation

#### imagedatagenerator 클래스 이용
 > https://keraskorea.github.io/posts/2018-10-24-little_data_powerful_model/

In [ ]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,      
        zoom_range=0.2,
        horizontal_flip=True,        
        fill_mode='nearest'
        )

In [ ]:
train_argu = datagen.flow(X_train, Y_train, batch_size=1)

In [ ]:
# fit parameters from data
datagen.fit(X_train)
# configure batch size and retrieve one batch of images
for X_batch, y_batch in datagen.flow(X_train, Y_train, batch_size=1):
	# create a grid of 3x3 images
	for i in range(0, 9):
		plt.subplot(330 + 1 + i)
		plt.imshow(X_batch[i].reshape(32, 32,3))
	# show the plot
	plt.show()
	break

NameError: ignored

#### 모델

In [ ]:
# with strategy.scope():
  # model
model = keras.models.Sequential([
            keras.layers.Conv2D(filters = 32, kernel_size=(5, 5), activation='relu', input_shape=(32,32,3)),
            keras.layers.MaxPool2D(pool_size=(3, 3)),
            keras.layers.Conv2D(filters = 64 , kernel_size=(3, 3), activation='relu'),
            keras.layers.MaxPool2D(pool_size=(2, 2)),
            keras.layers.Flatten(),
            keras.layers.Dense(256, activation='relu'),
            keras.layers.Dense(num_class, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics='accuracy'
)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 9, 9, 32)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               147712    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2

In [ ]:
# callback method

# checkpoint path
ckpt_name = 'CNN_ckpt.hdf5'
checkpoint_dir_path = os.path.join(prefix, 'checkpoint')
checkpoint_path = os.path.join(prefix, 'checkpoint', ckpt_name)

# check checkpoint paht
# if not exist, make dir
if not(os.path.exists(checkpoint_dir_path)):
  os.mkdir(checkpoint_dir_path)

callbacks_list = [
    # save model weights, at every epoch
    tf.keras.callbacks.ModelCheckpoint(
        filepath = checkpoint_path,
        monitor='val_loss',
        mode='min',
        save_weights_only=True,
        save_best_only=True
    ),
    # EarlyStopping, if val_loss is not improve five times
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        mode='min',
        verbose=1, 
        patience=5
    )
]

In [ ]:
hist = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=128), validation_data=(X_val,Y_val),
                 callbacks=callbacks_list,
                 epochs=20)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
1250/1250 [==============================] - 62s 26ms/step - loss: 1.8492 - accuracy: 0.3184 - val_loss: 1.5210 - val_accuracy: 0.4425
Epoch 2/20
1250/1250 [==============================] - 34s 27ms/step - loss: 1.6680 - accuracy: 0.3925 - val_loss: 1.4193 - val_accuracy: 0.4924
Epoch 3/20
1250/1250 [==============================] - 33s 26ms/step - loss: 1.5707 - accuracy: 0.4326 - val_loss: 1.3348 - val_accuracy: 0.5173
Epoch 4/20
1250/1250 [==============================] - 33s 26ms/step - loss: 1.5220 - accuracy: 0.4512 - val_loss: 1.2971 - val_accuracy: 0.5371
Epoch 5/20
1250/1250 [==============================] - 33s 26ms/step - loss: 1.4757 - accuracy: 0.4685 - val_loss: 1.2924 - val_accuracy: 0.5405
Epoch 6/20
1250/1250 [==============================] - 33s 27ms/step - loss: 1.4367 - accuracy: 0.4898 - val_loss: 1.2797 - val_accuracy: 0.5517
Epoch 7/20
1250/1250 [==============================] - 33s 27ms/step - loss: 1.4090 - accuracy: 0.4968 - val_loss: 1.2405 -

In [ ]:
# load last ckpt weight
model.load_weights(checkpoint_path)

In [ ]:
model.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 4ms/step - loss: 276.1352 - accuracy: 0.3849


[276.13519287109375, 0.3849000036716461]